# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmp1llpzqr2/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.3
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift
[3/4] Compiling SwiftCoreMLTools Activations.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalled

In [2]:
import Foundation
import TensorFlow
import SwiftCoreMLTools
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// !wget -O "../data/housing.csv" https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
    try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
}

# Data Ingestion

In [4]:
// TODO: Download housing.csv

let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

let trainFeatures = Array(dataFeatures[0..<numTrainRecords])
let trainLabels = Array(dataLabels[0..<numTrainRecords])
let testFeatures = Array(dataFeatures[numTrainRecords...])
let testLabels = Array(dataLabels[numTrainRecords...])

print("Training shapes [\(trainFeatures.count) \(trainFeatures[0].count)] [\(trainLabels.count) \(trainLabels[0].count)]")
print("Testing shapes  [\(testFeatures.count) \(testFeatures[0].count)] [\(testLabels.count) \(testLabels[0].count)]")

let xAllTrain = Array(trainFeatures.joined())
let yAllTrain = Array(trainLabels.joined())
let xAllTest = Array(testFeatures.joined())
let yAllTest = Array(testLabels.joined())

let XTrainDeNorm = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, 1]))
let XTestDeNorm = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, 1]))

Training shapes [405 13] [405 1]
Testing shapes  [101 13] [101 1]


# Normalize

In [5]:
let mean = XTrainDeNorm.mean(alongAxes: 0)
let std = XTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[  2.0137098,   14.197531,    9.523555, 0.086419754,  0.53213036,   6.3311296,    64.47929,
    4.1678762,    6.832099,   353.68396,    18.03163,   379.84735,   11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.28098345, 0.11449408,  0.7311985,  29.000755,  2.1797554,
    6.544165,  132.14561,  2.2173452,  40.494495,   6.852825]]


In [6]:
let XTrain = (XTrainDeNorm - mean)/std
let XTest = (XTestDeNorm - mean)/std

In [7]:
print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


# Model

In [8]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

# Training

In [9]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [10]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [11]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 659.6241, MAE: 23.741005
Epoch 2: MSE: 625.5328, MAE: 23.08691
Epoch 3: MSE: 592.6527, MAE: 22.41334
Epoch 4: MSE: 557.94104, MAE: 21.622292
Epoch 5: MSE: 516.56665, MAE: 20.700022
Epoch 6: MSE: 468.5644, MAE: 19.594685
Epoch 7: MSE: 416.28674, MAE: 18.299212
Epoch 8: MSE: 360.53748, MAE: 16.845097
Epoch 9: MSE: 302.2099, MAE: 15.201446
Epoch 10: MSE: 247.75775, MAE: 13.464666
Epoch 11: MSE: 199.15347, MAE: 11.697039
Epoch 12: MSE: 155.63498, MAE: 9.983671
Epoch 13: MSE: 118.799126, MAE: 8.506119
Epoch 14: MSE: 92.9789, MAE: 7.3978367
Epoch 15: MSE: 74.83897, MAE: 6.4943695
Epoch 16: MSE: 61.33531, MAE: 5.715657
Epoch 17: MSE: 51.241524, MAE: 5.145381
Epoch 18: MSE: 43.449398, MAE: 4.676491
Epoch 19: MSE: 38.19488, MAE: 4.311634
Epoch 20: MSE: 34.17106, MAE: 4.059272
Epoch 21: MSE: 31.189503, MAE: 3.8295565
Epoch 22: MSE: 29.63555, MAE: 3.7526898
Epoch 23: MSE: 28.161343, MAE: 3.5935056
Epoch 24: MSE: 26.871157, MAE: 3.5208468
Epoch 25: MSE: 25.648582, MAE: 3.436476
Epoch

Epoch 202: MSE: 4.896701, MAE: 1.4578366
Epoch 203: MSE: 5.017736, MAE: 1.4498426
Epoch 204: MSE: 4.6976337, MAE: 1.4293594
Epoch 205: MSE: 4.9879847, MAE: 1.457371
Epoch 206: MSE: 4.6656065, MAE: 1.42319
Epoch 207: MSE: 4.823667, MAE: 1.4445202
Epoch 208: MSE: 4.7245007, MAE: 1.4296492
Epoch 209: MSE: 4.7553816, MAE: 1.4230783
Epoch 210: MSE: 4.6435695, MAE: 1.4381629
Epoch 211: MSE: 4.789824, MAE: 1.4378924
Epoch 212: MSE: 4.6190934, MAE: 1.4145122
Epoch 213: MSE: 4.6233773, MAE: 1.4112384
Epoch 214: MSE: 4.602334, MAE: 1.3973823
Epoch 215: MSE: 4.4955444, MAE: 1.4033589
Epoch 216: MSE: 4.690001, MAE: 1.4301164
Epoch 217: MSE: 4.636393, MAE: 1.4009197
Epoch 218: MSE: 4.4167647, MAE: 1.3944002
Epoch 219: MSE: 4.769875, MAE: 1.3790128
Epoch 220: MSE: 4.3899565, MAE: 1.3958647
Epoch 221: MSE: 4.497045, MAE: 1.3972646
Epoch 222: MSE: 4.411573, MAE: 1.3705394
Epoch 223: MSE: 4.5346518, MAE: 1.4144504
Epoch 224: MSE: 4.463779, MAE: 1.3987122
Epoch 225: MSE: 4.300244, MAE: 1.363907
Epoch 22

Epoch 401: MSE: 2.5326395, MAE: 1.0498567
Epoch 402: MSE: 2.3941672, MAE: 1.032579
Epoch 403: MSE: 2.597246, MAE: 1.0510337
Epoch 404: MSE: 2.44801, MAE: 1.056702
Epoch 405: MSE: 2.4724965, MAE: 1.0574672
Epoch 406: MSE: 2.3534544, MAE: 1.0333574
Epoch 407: MSE: 2.3328376, MAE: 1.0694798
Epoch 408: MSE: 2.2715514, MAE: 1.0551811
Epoch 409: MSE: 2.67068, MAE: 1.0950396
Epoch 410: MSE: 2.3202462, MAE: 1.0102298
Epoch 411: MSE: 2.2100372, MAE: 1.0502447
Epoch 412: MSE: 2.5493753, MAE: 1.0837442
Epoch 413: MSE: 2.4105608, MAE: 1.0463662
Epoch 414: MSE: 2.459645, MAE: 1.0193797
Epoch 415: MSE: 2.4246216, MAE: 1.0418363
Epoch 416: MSE: 2.352205, MAE: 1.0687385
Epoch 417: MSE: 2.244335, MAE: 1.0166111
Epoch 418: MSE: 2.422757, MAE: 1.0961397
Epoch 419: MSE: 2.4088466, MAE: 1.0484474
Epoch 420: MSE: 2.2856061, MAE: 1.0402004
Epoch 421: MSE: 2.2698715, MAE: 1.0278957
Epoch 422: MSE: 2.5329678, MAE: 1.0536726
Epoch 423: MSE: 2.3145902, MAE: 1.037894
Epoch 424: MSE: 2.3879187, MAE: 1.0095361
Epoc

# Test

In [12]:
Context.local.learningPhase = .inference

let prediction = model(XTest)

let predictionMse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let predictionMae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(predictionMse), MAE: \(predictionMae)")

MSE: 1.8502706, MAE: 0.10506772


In [13]:
print(XTrain[0], YTrain[0], model(XTrain[0].reshaped(to: TensorShape([1, 13]))))

print(XTest[0], YTest[0], model(XTest[0].reshaped(to: TensorShape([1, 13]))))
print(XTest[1], YTest[1], model(XTest[1].reshaped(to: TensorShape([1, 13]))))
print(XTest[17], YTest[17], model(XTest[17].reshaped(to: TensorShape([1, 13]))))

[ -0.30846816,   0.15054052,   -1.1039964,  -0.30756173,   0.05126577,   0.33352128,
  0.024851447, -0.035726987,  -0.89119065,  -0.43651816,   -1.2319369,   0.42111015,
  -0.93603975] [24.0] [[23.943727]]
[  10.127699, -0.56208307,   1.3125796, -0.30756173,   1.4050479,  -0.8863933,   1.2248203,
  -1.2581577,   2.6233904,   2.3634233,   0.9779132,  0.12650238,   1.6906141] [5.0] [[11.277078]]
[  2.8739426, -0.56208307,   1.3125796, -0.30756173,    1.108089,  -2.9993627,   1.2248203,
  -1.3716108,   2.6233904,   2.3634233,   0.9779132, -0.23774466,   1.7431473] [11.9] [[12.968968]]
[   1.541963, -0.56208307,   1.3125796, -0.30756173,   0.7150559, -0.93426037,   0.7972452,
  -1.0169381,   2.6233904,   2.3634233,   0.9779132,   -2.180478,  0.39479825] [20.8] [[32.2526]]


# Export

In [14]:
print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

[13, 64] [64, 32] [32, 1]
[64] [32] [1]


In [15]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
    Input(name: "input", shape: [13], featureType: .Double)
    Output(name: "output", shape: [1], featureType: .Double)
    NeuralNetwork {
        InnerProduct(name: "dense1",
                     input: ["input"],
                     output: ["outDense1"],
                     weights: model.layer1.weight.flattened().scalars,
                     bias: model.layer1.bias.flattened().scalars,
                     inputChannels: 13,
                     outputChannels: 64,
                     updatable: false)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weights: model.layer2.weight.flattened().scalars,
                     bias: model.layer2.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32,
                     updatable: false)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weights: model.layer3.weight.flattened().scalars,
                     bias: model.layer3.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1,
                     updatable: false)
    }
}

In [16]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))